In [1]:
!pip install tensorflow==2.4.1

     -------------------------------------- 370.7/370.7 MB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 13.2 MB/s eta 0:00:00
     ------------------------------------- 462.0/462.0 KB 14.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorf

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Iris_Data') 

# Actions that we try to detect
actions = np.array(['yes', 'no' ])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [3]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [8]:
import argparse

import cv2
import mediapipe as mp
import numpy as np
import csv


from custom.iris_lm_depth import from_landmarks_to_depth
from videosource import FileSource, WebcamSource

mp_face_mesh = mp.solutions.face_mesh

left_eye_landmarks_id = np.array([33, 133])
right_eye_landmarks_id = np.array([362, 263])

dist_coeff = np.zeros((4, 1))

YELLOW = (0, 255, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
RED = (0, 0, 255)
SMALL_CIRCLE_SIZE = 1
LARGE_CIRCLE_SIZE = 2


LEFT_EYE_LANDMARKS_ID = np.array([33, 133])
RIGHT_EYE_LANDMARKS_ID = np.array([362, 263])

POINTS_IDX = [33, 133, 362, 263]
POINTS_IDX = list(set(POINTS_IDX))
POINTS_IDX.sort()


def __add_landmark_to_df(landmark, landmark_idx, df_headers, df_values):
    """Helper function that adds a landmark to the dataframe"""

    df_headers.append("x{}".format(landmark_idx))
    df_headers.append("y{}".format(landmark_idx))
    df_headers.append("z{}".format(landmark_idx))

    df_values.append(landmark[0])
    df_values.append(landmark[1])
    df_values.append(landmark[2])


def main1():
    FIRST_TIME = True
    frame_height, frame_width = (720, 1280)
    source = WebcamSource(width=frame_width, height=frame_height)
    image_size = (frame_width, frame_height)
    count = 0

    # pseudo camera internals
    focal_length = frame_width

    landmarks = None
    smooth_left_depth = -1
    smooth_right_depth = -1
    smooth_factor = 0.1

    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as face_mesh:
        for action in actions:
            # Loop through sequences aka videos
            for sequence in range(no_sequences):
                # Loop through video length aka sequence length
                for idx, (frame, frame_rgb) in enumerate(source):
                    # only go 30 times
                    count = count + 1
                    print(count)
                    if count == 30:
                        count = 0
                        break
#                     for idx, (frame, frame_rgb) in enumerate(source):
                    results = face_mesh.process(frame_rgb)
                    multi_face_landmarks = results.multi_face_landmarks

                    if multi_face_landmarks:
                        face_landmarks = results.multi_face_landmarks[0]
                        landmarks = np.array(
                            [(lm.x, lm.y, lm.z)
                             for lm in face_landmarks.landmark]
                        )
                        landmarks = landmarks.T

                        (
                            left_depth,
                            left_iris_size,
                            left_iris_landmarks,
                            left_eye_contours,
                        ) = from_landmarks_to_depth(
                            frame_rgb,
                            landmarks[:, left_eye_landmarks_id],
                            image_size,
                            is_right_eye=False,
                            focal_length=focal_length,
                        )

                        (
                            right_depth,
                            right_iris_size,
                            right_iris_landmarks,
                            right_eye_contours,
                        ) = from_landmarks_to_depth(
                            frame_rgb,
                            landmarks[:, right_eye_landmarks_id],
                            image_size,
                            is_right_eye=True,
                            focal_length=focal_length,
                        )

                        if smooth_right_depth < 0:
                            smooth_right_depth = right_depth
                        else:
                            smooth_right_depth = (
                                smooth_right_depth * (1 - smooth_factor)
                                + right_depth * smooth_factor
                            )

                        if smooth_left_depth < 0:
                            smooth_left_depth = left_depth
                        else:
                            smooth_left_depth = (
                                smooth_left_depth * (1 - smooth_factor)
                                + left_depth * smooth_factor
                            )

                    if landmarks is not None:

                        landmark_idx = 0
                        df_headers = []
                        df_values = []

                        # add eye contours to dataframe
                        eye_landmarks = np.concatenate(
                            [
                                right_eye_contours[0:17],
                                left_eye_contours[0:17],
                            ]
                        )

                        # add iris landmarks to dataframe
                        iris_landmarks = np.concatenate(
                            [
                                right_iris_landmarks,
                                left_iris_landmarks,
                            ]
                        )
                        for landmark in iris_landmarks:

                            __add_landmark_to_df(
                                landmark, landmark_idx, df_headers, df_values)

                            landmark_idx += 1

                        for landmark in eye_landmarks:

                            __add_landmark_to_df(
                                landmark, landmark_idx, df_headers, df_values)

                            landmark_idx += 1

                        # add subset of facemesh to dataframe
                        for ii in POINTS_IDX:

                            landmark = (landmarks[0, ii],
                                        landmarks[1, ii], landmarks[2, ii])
                            __add_landmark_to_df(
                                landmark, landmark_idx, df_headers, df_values)

                            landmark_idx += 1

                         # Export to CSV
#                         with open(outputfilepath, mode='a', newline='') as f:
#                             csv_writer = csv.writer(
#                                 f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                             if (FIRST_TIME):
#                                 csv_writer.writerow(df_headers)
#                                 FIRST_TIME = False
#                             csv_writer.writerow(df_values)

                        # draw subset of facemesh
                        for ii in POINTS_IDX:
                            pos = (np.array(image_size) *
                                   landmarks[:2, ii]).astype(np.int32)
                            frame = cv2.circle(frame, tuple(
                                pos), LARGE_CIRCLE_SIZE, GREEN, -1)

                        # draw eye contours
                        eye_landmarks = np.concatenate(
                            [
                                right_eye_contours[0:17],
                                left_eye_contours[0:17],
                            ]
                        )
                        for landmark in eye_landmarks:
                            pos = (np.array(image_size) *
                                   landmark[:2]).astype(np.int32)
                            frame = cv2.circle(frame, tuple(
                                pos), SMALL_CIRCLE_SIZE, RED, -1)

                        # draw iris landmarks
                        iris_landmarks = np.concatenate(
                            [
                                right_iris_landmarks[0:3],
                                left_iris_landmarks[0:5],
                            ]
                        )
                        for landmark in iris_landmarks:
                            pos = (np.array(image_size) *
                                   landmark[:2]).astype(np.int32)
                            frame = cv2.circle(frame, tuple(
                                pos), SMALL_CIRCLE_SIZE, YELLOW, -1)

                        # write depth values into frame
                        depth_string = "{:.2f}cm, {:.2f}cm".format(
                            smooth_left_depth / 10, smooth_right_depth / 10
                        )
                        frame = cv2.putText(
                            frame,
                            depth_string,
                            (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            GREEN,
                            2,
                            cv2.LINE_AA,
                        )

                     # NEW Apply wait logic
                        if count == 1:
                            cv2.putText(frame, 'STARTING COLLECTION', (120, 200),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                            cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            # Show to screen
                            source.show(frame)
                            cv2.waitKey(2000)
                        else:
                            cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            source.show(frame)

                        keypoints = np.array(df_values)
                        print(keypoints)
                        npy_path = os.path.join(
                            DATA_PATH, action, str(sequence), str(count))
                        np.save(npy_path, keypoints)


main1()


1
[ 0.390767    0.42991364 -0.12589774  0.38336391  0.42905387 -0.12541634
  0.39149466  0.4168939  -0.12632529  0.39825062  0.43084192 -0.12646197
  0.39009507  0.44300007 -0.12581296  0.30943778  0.46298145  0.12049884
  0.31746154  0.45941741  0.1203717   0.30747495  0.4486186   0.11996667
  0.30117918  0.4666245   0.11989538  0.31137595  0.47733434  0.12020083
  0.40548366  0.4314273   0.16077085  0.40353694  0.43656206  0.12216453
  0.40127908  0.44019467  0.08851694  0.39763892  0.44304128  0.04362081
  0.39178909  0.44418503  0.00294825  0.3857524   0.4431295  -0.02208166
  0.38030178  0.44083769 -0.02733726  0.37704295  0.43962983 -0.02122288
  0.37559579  0.43854094 -0.02907451  0.40469812  0.42829768  0.11047252
  0.40341094  0.42483474  0.06847804  0.40040138  0.4205799   0.02356813
  0.39448848  0.4184983  -0.02098023  0.38791608  0.42015025 -0.04555384
  0.38175893  0.42591283 -0.04784837  0.3775704   0.43368941 -0.03719203
  0.40747601  0.43095213  0.18937053  0.29312252 

[ 0.39033983  0.42463071 -0.07488684  0.38287769  0.42399917 -0.07488237
  0.39089002  0.41185256 -0.07447655  0.39791743  0.42512471 -0.0754238
  0.3897968   0.43737812 -0.07560655  0.31003839  0.45478376  0.134298
  0.31801133  0.45162039  0.1341978   0.30826847  0.44081364  0.1337686
  0.30172899  0.45803838  0.13366322  0.31180345  0.46888626  0.13428046
  0.40604353  0.42482994  0.1298524   0.40398507  0.42965579  0.09619866
  0.40169427  0.43320874  0.06727476  0.39811793  0.43617846  0.03015972
  0.39243864  0.43764523 -0.0011112   0.38666038  0.43673813 -0.01696819
  0.38123676  0.434354   -0.0160424   0.3778878   0.43296471 -0.00612842
  0.37639072  0.43183505 -0.01063088  0.40487418  0.42257532  0.08456176
  0.4033408   0.41991861  0.04733544  0.40026578  0.41648429  0.00977972
  0.39464734  0.41453286 -0.02523797  0.38848719  0.41554613 -0.0413085
  0.38242606  0.4203051  -0.0368997   0.37827623  0.42727999 -0.0218831
  0.40840766  0.42426923  0.15367232  0.293531    0.46403

10
[ 3.86299905e-01  4.36005105e-01 -9.79052037e-02  3.78766559e-01
  4.35464043e-01 -9.74019170e-02  3.86578205e-01  4.22782940e-01
 -9.76439267e-02  3.93985215e-01  4.36343945e-01 -9.83618051e-02
  3.86034396e-01  4.49174160e-01 -9.81753990e-02  3.06629235e-01
  4.62292624e-01  1.48069948e-01  3.14202720e-01  4.58974849e-01
  1.48338795e-01  3.04845509e-01  4.48915158e-01  1.47649735e-01
  2.98805255e-01  4.65622727e-01  1.47720188e-01  3.08417001e-01
  4.75742139e-01  1.48292378e-01  4.03194857e-01  4.33451928e-01
  1.53154507e-01  4.01425663e-01  4.38394022e-01  1.14791229e-01
  3.99284157e-01  4.42022207e-01  8.15129802e-02  3.95755348e-01
  4.45130751e-01  3.82291116e-02  3.89896578e-01  4.46730063e-01
 -2.94414582e-04  3.83800426e-01  4.45647981e-01 -2.37402767e-02
  3.78153431e-01  4.42984221e-01 -2.85116155e-02  3.74790570e-01
  4.41333453e-01 -2.22727545e-02  3.73161088e-01  4.39949883e-01
 -2.98541691e-02  4.02147320e-01  4.30959744e-01  1.04446135e-01
  4.00705561e-01  4.28

14
[ 3.82870379e-01  4.37659147e-01 -7.37903789e-02  3.75164174e-01
  4.36872260e-01 -7.36725777e-02  3.83319688e-01  4.24408796e-01
 -7.31180832e-02  3.90769121e-01  4.38429477e-01 -7.36069754e-02
  3.82449597e-01  4.50853146e-01 -7.39030316e-02  3.01913053e-01
  4.63210196e-01  1.49272308e-01  3.09135199e-01  4.60473273e-01
  1.49390638e-01  3.00411102e-01  4.50427453e-01  1.48196205e-01
  2.94462001e-01  4.66089365e-01  1.49020493e-01  3.03431952e-01
  4.76254887e-01  1.50091782e-01  4.02305216e-01  4.35374334e-01
  1.55138806e-01  4.00265524e-01  4.40580357e-01  1.17072962e-01
  3.97931871e-01  4.44440858e-01  8.38296339e-02  3.94141605e-01
  4.47821580e-01  4.03010026e-02  3.87997276e-01  4.49630859e-01
  1.35770557e-03  3.81513885e-01  4.48587079e-01 -2.17685606e-02
  3.75493369e-01  4.45729399e-01 -2.50682253e-02  3.71795289e-01
  4.43914048e-01 -1.72472782e-02  3.70036511e-01  4.42318466e-01
 -2.42515095e-02  4.01141295e-01  4.32526265e-01  1.05102845e-01
  3.99547526e-01  4.29

[ 3.79152724e-01  4.42793459e-01 -2.82787401e-02  3.71377566e-01
  4.41609780e-01 -2.83508003e-02  3.79821181e-01  4.29559485e-01
 -2.85647511e-02  3.87090957e-01  4.43988546e-01 -2.81759873e-02
  3.78528249e-01  4.55936188e-01 -2.83515342e-02  2.97121906e-01
  4.66667334e-01  1.47314399e-01  3.04448119e-01  4.63525030e-01
  1.47366434e-01  2.95379433e-01  4.53945462e-01  1.46123856e-01
  2.89522341e-01  4.69768365e-01  1.48055211e-01  2.98890743e-01
  4.79412821e-01  1.47365153e-01  3.98263508e-01  4.39786508e-01
  1.67906627e-01  3.96262240e-01  4.45348559e-01  1.28131419e-01
  3.93934223e-01  4.49389760e-01  9.28846225e-02  3.90086347e-01
  4.52872266e-01  4.67638634e-02  3.83689392e-01  4.54699707e-01
  4.12986381e-03  3.76933548e-01  4.53683572e-01 -2.31308471e-02
  3.70586425e-01  4.50905779e-01 -2.94095855e-02  3.66672792e-01
  4.49222904e-01 -2.34266296e-02  3.64792663e-01  4.47601403e-01
 -3.16670537e-02  3.97115332e-01  4.36614153e-01  1.15506895e-01
  3.95482993e-01  4.33227

...

[ 0.44310079  0.37152061  0.06161532  0.43547865  0.37088476  0.06128521
  0.44331754  0.35832979  0.06107534  0.45095273  0.37205651  0.0611783
  0.44287155  0.38462178  0.06118161  0.36497657  0.37589149  0.06339727
  0.37178844  0.37382209  0.06303656  0.36368935  0.36387379  0.06229376
  0.35793715  0.37803857  0.06256962  0.3662623   0.38800854  0.06348538
  0.45964466  0.37153179  0.08641954  0.45689533  0.37661215  0.05873611
  0.4538215   0.37996575  0.03668599  0.44952332  0.38273998  0.01066151
  0.44327232  0.38409503 -0.00778179  0.43692443  0.38300794 -0.00893769
  0.43063907  0.38050268  0.0048293   0.42645723  0.3792684   0.02256935
  0.42453037  0.37802448  0.0228006   0.45801237  0.36880141  0.04737735
  0.45579932  0.36562165  0.01695128  0.45187642  0.36175498 -0.01148253
  0.4455779   0.35973739 -0.03177219  0.43898758  0.36091105 -0.03272936
  0.43216877  0.3661188  -0.01689738  0.42693343  0.37339693  0.0061747
  0.46245993  0.37088556  0.1034077   0.35091485  0.3

29
[ 0.44344551  0.37222724  0.05026542  0.4356872   0.37160423  0.04975749
  0.44360977  0.35877098  0.04954913  0.45138673  0.37269735  0.04990869
  0.44325871  0.3855112   0.04969552  0.36532866  0.37599179  0.07620845
  0.37227242  0.37358659  0.07584829  0.36387628  0.36384762  0.07482896
  0.35814324  0.37849668  0.07537008  0.3667619   0.3882949   0.07634427
  0.45963787  0.37222007  0.08953025  0.45699505  0.37703964  0.06153667
  0.45405891  0.38032698  0.03888216  0.44992301  0.38316952  0.01216095
  0.44378877  0.38475228 -0.00742845  0.43746091  0.3838906  -0.00995119
  0.43114089  0.38149601  0.00277389  0.42691844  0.38027946  0.01989296
  0.42496231  0.37903515  0.01970886  0.45795625  0.3698307   0.04940965
  0.45569769  0.36702833  0.01805851  0.45178947  0.36358756 -0.01139225
  0.44569225  0.36178009 -0.03330376  0.43927734  0.36284433 -0.03557819
  0.43252601  0.3676108  -0.01991841  0.42731082  0.37453431  0.00317708
  0.46256891  0.37138297  0.10701922  0.35122591

7
[ 4.43649727e-01  3.71294218e-01  4.62758839e-02  4.35885996e-01
  3.70759344e-01  4.58808616e-02  4.43803194e-01  3.58017365e-01
  4.57209945e-02  4.51619273e-01  3.71702756e-01  4.58535999e-02
  4.43485197e-01  3.84503894e-01  4.56957556e-02  3.65476471e-01
  3.75430838e-01  6.99801221e-02  3.72729278e-01  3.73034647e-01
  6.96606040e-02  3.64046893e-01  3.62872505e-01  6.85805008e-02
  3.57984161e-01  3.77931680e-01  6.91530034e-02  3.66888595e-01
  3.88127825e-01  7.00097233e-02  4.59923387e-01  3.71285105e-01
  8.89482573e-02  4.57334560e-01  3.76356999e-01  6.16372004e-02
  4.54392415e-01  3.79866706e-01  3.95285413e-02  4.50210044e-01
  3.82942396e-01  1.32270455e-02  4.43981317e-01  3.84681956e-01
 -6.31472794e-03  4.37572268e-01  3.83832889e-01 -9.19064227e-03
  4.31162950e-01  3.81356955e-01  3.06727597e-03  4.26905154e-01
  3.80151659e-01  1.94348004e-02  4.24960060e-01  3.78940286e-01
  1.91079825e-02  4.58290207e-01  3.68790759e-01  4.91616055e-02
  4.56063789e-01  3.658

15
[ 4.43117642e-01  3.72928826e-01  1.33697689e-02  4.35755858e-01
  3.72398048e-01  1.32636353e-02  4.43275899e-01  3.60508431e-01
  1.31816119e-02  4.50568822e-01  3.73314942e-01  1.29857287e-02
  4.42925233e-01  3.85174296e-01  1.30961854e-02  3.64648482e-01
  3.77812274e-01  7.07560927e-02  3.71004194e-01  3.75673485e-01
  7.03472644e-02  3.63342860e-01  3.66438521e-01  6.96392432e-02
  3.58165196e-01  3.80054622e-01  7.04124421e-02  3.65961775e-01
  3.89307928e-01  7.12060481e-02  4.59189802e-01  3.73581468e-01
  9.37942713e-02  4.56684315e-01  3.77740230e-01  6.48347214e-02
  4.53819305e-01  3.80656475e-01  4.13171351e-02  4.49792960e-01
  3.83444866e-01  1.39249070e-02  4.43681520e-01  3.85250775e-01
 -7.16730068e-03  4.37377533e-01  3.84802310e-01 -1.14598731e-02
  4.31088504e-01  3.82772737e-01 -7.92500432e-05  4.26981243e-01
  3.81776667e-01  1.63441673e-02  4.25056010e-01  3.80740839e-01
  1.58339143e-02  4.57384783e-01  3.71959972e-01  5.19359186e-02
  4.54903239e-01  3.70

23
[ 0.44131391  0.36879964  0.03340811  0.43368277  0.36839006  0.03323409
  0.44138305  0.35604365  0.0331412   0.44912813  0.36906349  0.03309121
  0.4412621   0.38147046  0.03338791  0.36242912  0.37826979  0.06211838
  0.36877614  0.37608819  0.06179566  0.36108247  0.366794    0.06115695
  0.35590746  0.38056347  0.06180905  0.36377529  0.38985376  0.06253996
  0.45687467  0.36872254  0.08097471  0.45430379  0.37322287  0.05533637
  0.45139682  0.37641075  0.03451854  0.44732153  0.37933095  0.01043563
  0.44129716  0.38119704 -0.0067387   0.43511288  0.38075287 -0.00761475
  0.42888421  0.37877046  0.00571728  0.42478969  0.3778864   0.02246289
  0.42294163  0.37689131  0.02319843  0.45512996  0.36671914  0.04221023
  0.45280075  0.36436459  0.0127045   0.44885583  0.3616375  -0.01515628
  0.44274192  0.36057377 -0.03490151  0.43641689  0.36192097 -0.03507024
  0.42990757  0.36642111 -0.01727893  0.42505297  0.37278153  0.00594733
  0.45987666  0.36773296  0.09724916  0.34944318